# Deep Learning

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.regularizers import l1 as l1_regularizer, l2 as l2_regularizer
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import pandas as pd

# RMSLE 손실 함수 정의
def rmsle(y_true, y_pred):
    y_true = tf.maximum(tf.cast(y_true, tf.float32), 0)
    y_pred = tf.maximum(tf.cast(y_pred, tf.float32), 0)
    squared_error = tf.square(tf.math.log1p(y_pred) - tf.math.log1p(y_true))
    return tf.sqrt(tf.reduce_mean(squared_error))

# 모델 생성 함수 정의
def create_model(learning_rate, l1_reg, l2_reg):
    input_layer = tf.keras.Input(shape=(len(train_x.columns),))
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=l1_regularizer(l1_reg))(x)
    x = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2_regularizer(l2_reg))(x)
    output_layer = tf.keras.layers.Dense(1)(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss=rmsle,
                  metrics=[rmsle])
    return model

def loss_fn(y_true, y_pred):
    return rmsle(y_true, y_pred) #차이 손실함수 반환

def metric_fn(y_true, y_pred):
    return rmsle(y_true, y_pred)

In [ ]:
# KerasRegressor 래퍼를 사용
model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=32, verbose=0)

# 하이퍼파라미터 그리드 정의
param_grid = {
    'learning_rate': [0.1, 0.01, 0.001, 0.0001],
    'l1_reg': [0.01, 0.001, 0.0001],
    'l2_reg': [0.01, 0.001, 0.0001],
    'batch_size': [32, 64, 128]
}

# 사용자 정의 스코어링 함수 (RMSLE)
def rmsle_scorer(y_true, y_pred):
    y_true = np.maximum(y_true, 0)
    y_pred = np.maximum(y_pred, 0)
    squared_error = np.square(np.log1p(y_pred) - np.log1p(y_true))
    return np.sqrt(np.mean(squared_error))

# GridSearchCV 생성
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    scoring=make_scorer(rmsle_scorer, greater_is_better=False), 
                    cv=3)

# 그리드 서치 실행
grid_result = grid.fit(train_x.astype('float32'), train_y.astype('float32'))

# 결과 출력
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# 결과를 데이터프레임으로 변환 후 CSV 파일로 저장
results_df = pd.DataFrame(grid_result.cv_results_)
results_df.to_csv('hyperparameter_tuning_results.csv', index=False)

# Grid Search 돌려서 결과 확인하기

## 아래 코드로 최종 Grid Search찾으면 모델 돌려서 결과보기

In [ ]:
import tensorflow as tf
from tensorflow.keras.regularizers import l1 as l1_regularizer, l2 as l2_regularizer


def create_optimized_model():
    input_layer = tf.keras.Input(shape=(len(train_x.columns),))
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=l1_regularizer(0.001))(x)
    x = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=l2_regularizer(0.001))(x)
    output_layer = tf.keras.layers.Dense(1)(x)
    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=loss_fn,
                  metrics=[metric_fn])
    return model

# 최적화된 모델 생성
optimized_model = create_optimized_model()

# 모델 학습
history = optimized_model.fit(train_x.astype('float32'), train_y.astype('float32'),
                              epochs=100, batch_size=32, verbose=1,
                              validation_split=0.1, callbacks=callbacks_list)
